In [1]:
import pandas as pd
import numpy as np

## Load CSV data (Dataset A)

In [2]:
# csv_path = '../data/learning_traces.13m.csv'
csv_path = r'/Volumes/andy_SSD/KUL/Datathon_Duolingo/Dataset A/learning_traces.13m.csv'

df = pd.read_csv(csv_path)

## Extract the pt data

In [3]:
df_pt = df[df['learning_language'] == 'pt'].copy()
df_pt

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
64,1.0,1362082493,1469,u:g3WM,pt,en,57408f89412af98111a2f87c0ab41b22,tu/tu<prn><tn><p2><mf><sg>,48,48,1,1
65,0.5,1362082493,1469,u:g3WM,pt,en,8414835cb39e4315146a59fefdd6d1c6,tem/ter<vblex><pri><p3><sg>,2,2,2,1
66,1.0,1362082493,1469,u:g3WM,pt,en,ecc3feb8e53ce936cef181dd54e7aaca,temos/ter<vblex><pri><p1><pl>,1,1,1,1
67,1.0,1362082493,2184,u:g3WM,pt,en,8d28ba0fa188f1847571467189846dda,tua/teu<det><pos><f><sg>,4,3,1,1
68,1.0,1362082493,1469,u:g3WM,pt,en,4b3613233b3fede2e3e92ac2ef752bf6,leão/leão<n><m><sg>,9,8,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
12852940,1.0,1363104812,604744,u:h3b5,pt,en,09cd6338773c2b99587b336a6141b185,gatas/gato<n><f><pl>,13,12,1,1
12852941,1.0,1363104812,285948,u:h3b5,pt,en,7ab5ef9586b79b2079fcce99fadd811f,cão/cão<n><m><sg>,8,7,1,1
12852942,1.0,1363104812,841,u:h3b5,pt,en,ba634cde4013c8ff4eee5892e92b3e5b,é/ser<vbser><pri><p3><sg>,54,49,1,1
12852943,1.0,1363104812,841,u:h3b5,pt,en,630b709686520fc4f0d0e6077df220c3,bebe/beber<vblex><pri><p3><sg>,27,27,2,2


## Featrue Engineering

In [4]:
# Extract the word by column: lexeme_string
df_pt['word_only'] = df_pt['lexeme_string'].str.split('<').str[0]
print(df_pt[['lexeme_string', 'word_only']].head())

                    lexeme_string  word_only
64     tu/tu<prn><tn><p2><mf><sg>      tu/tu
65    tem/ter<vblex><pri><p3><sg>    tem/ter
66  temos/ter<vblex><pri><p1><pl>  temos/ter
67       tua/teu<det><pos><f><sg>    tua/teu
68            leão/leão<n><m><sg>  leão/leão


In [5]:
# Process some features in the dataframe
df_pt['log_delta'] = np.log1p(df_pt['delta'])
df_pt['history_acc_rate'] = df_pt['history_correct'] / (df_pt['history_seen'] + 0.0001)
df_pt['grammar_tags'] = df_pt['lexeme_string'].str.extract(r'(<.*>)')
df_pt.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct,word_only,log_delta,history_acc_rate,grammar_tags
64,1.0,1362082493,1469,u:g3WM,pt,en,57408f89412af98111a2f87c0ab41b22,tu/tu<prn><tn><p2><mf><sg>,48,48,1,1,tu/tu,7.293018,0.999998,<prn><tn><p2><mf><sg>
65,0.5,1362082493,1469,u:g3WM,pt,en,8414835cb39e4315146a59fefdd6d1c6,tem/ter<vblex><pri><p3><sg>,2,2,2,1,tem/ter,7.293018,0.999950,<vblex><pri><p3><sg>
66,1.0,1362082493,1469,u:g3WM,pt,en,ecc3feb8e53ce936cef181dd54e7aaca,temos/ter<vblex><pri><p1><pl>,1,1,1,1,temos/ter,7.293018,0.999900,<vblex><pri><p1><pl>
67,1.0,1362082493,2184,u:g3WM,pt,en,8d28ba0fa188f1847571467189846dda,tua/teu<det><pos><f><sg>,4,3,1,1,tua/teu,7.689371,0.749981,<det><pos><f><sg>
68,1.0,1362082493,1469,u:g3WM,pt,en,4b3613233b3fede2e3e92ac2ef752bf6,leão/leão<n><m><sg>,9,8,1,1,leão/leão,7.293018,0.888879,<n><m><sg>


In [6]:
# Using 'lexeme_id' to get 'lexeme_code' for each word 
df_pt['lexeme_code'], uniques = pd.factorize(df_pt['lexeme_id'])

# Calculate 'lexeme_avg_recall' by averange their 'p_recall'
lexeme_difficulty = df_pt.groupby('lexeme_code')['p_recall'].mean()
df_pt['lexeme_avg_recall'] = df_pt['lexeme_code'].map(lexeme_difficulty) # Correct answer rate for single words

# # Define a new index: evaluate how difficulty is the word: 1 - 'lexeme_avg_recall'
# df_pt['word_difficulty_index'] = 1 - df_pt['lexeme_avg_recall'] # The more difficult the word, the higher the word_difficulty_index

# user’s historical correctness for each lexeme weighted by lexeme difficulty
df_pt['user_ability_index'] = df_pt['history_acc_rate'] ** df_pt['lexeme_avg_recall']

df_pt[['lexeme_id', 'lexeme_code', 'lexeme_avg_recall', 'user_ability_index']].head(10)

,lexeme_id,lexeme_code,lexeme_avg_recall,user_ability_index
64,57408f89412af98111a2f87c0ab41b22,0,0.943455,0.999998
65,8414835cb39e4315146a59fefdd6d1c6,1,0.878241,0.999956
66,ecc3feb8e53ce936cef181dd54e7aaca,2,0.919952,0.999908
67,8d28ba0fa188f1847571467189846dda,3,0.937195,0.763656
68,4b3613233b3fede2e3e92ac2ef752bf6,4,0.917828,0.897525
69,2e327d99285380dc37273580705b9f85,5,0.711111,0.999929
70,213ad43df9831b7b21ca1d7d79f5d107,6,0.941338,0.999997
71,41db134d8af1c0610b0d926d8a53d01a,7,0.895066,0.849417
72,6e820e73c9936c1c90e20a174f3dd1ad,8,0.779438,0.999961
73,38ec3b04540c300424b556dee6e5dbe5,9,0.922902,0.999969


## User's Fingerprint

In [7]:
# Calculate basic statistics by User-level (Max history, Vocab size, Time range)
user_base = df_pt.groupby('user_id').agg(
    max_history_seen=('history_seen', 'max'),
    vocab_size=('lexeme_id', 'nunique'),
    ts_min=('timestamp', 'min'),
    ts_max=('timestamp', 'max')
).reset_index()

# Calculate Feature Learning Speed
user_base['learning_speed'] = user_base['vocab_size'] / ((user_base['ts_max'] - user_base['ts_min'] + 1) / 86400)

# Organize the DataFrame columns
df_user_base = user_base[[
    'user_id', 
    'max_history_seen',    # Experience index (maximum number of practice sessions)
    'vocab_size',          # User's age index (number of words)
    'learning_speed'       # Learning speed metric (new words/day)
]].copy()

# Handling extreme values
df_user_base['learning_speed'] = np.log1p(df_user_base['learning_speed'])
df_user_base.head()

,user_id,max_history_seen,vocab_size,learning_speed
0,u:0X2,15,9,13.563969
1,u:0b,12,12,3.629236
2,u:0xw,20,108,2.633296
3,u:1EH,14,52,2.580331
4,u:1gx,15,71,7.584663


In [8]:
# Mapping 'history_seen' to each lexeme
df_lexeme_seen = df_pt.pivot_table(
    index='user_id', 
    columns='lexeme_code', 
    values='history_seen',
    aggfunc='max' # If there are multiple records for the same character, take the maxiunm
)

# Rename the columns to: lexeme_0_seen, lexeme_1_seen...
df_lexeme_seen.columns = [f'lexeme_{int(col)}_seen' for col in df_lexeme_seen.columns]

# Handling missing values
df_lexeme_seen = df_lexeme_seen.fillna(0)
df_lexeme_seen.head()

,lexeme_0_seen,lexeme_1_seen,lexeme_2_seen,lexeme_3_seen,lexeme_4_seen,lexeme_5_seen,lexeme_6_seen,lexeme_7_seen,lexeme_8_seen,lexeme_9_seen,...,lexeme_2805_seen,lexeme_2806_seen,lexeme_2807_seen,lexeme_2808_seen,lexeme_2809_seen,lexeme_2810_seen,lexeme_2811_seen,lexeme_2812_seen,lexeme_2813_seen,lexeme_2814_seen
user_id,,,,,,,,,,,,,,,,,,,,,
u:0X2,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u:0b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u:0xw,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u:1EH,12.0,0.0,0.0,0.0,7.0,0.0,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u:1gx,15.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Mapping 'user_ability_index' to each lexeme
df_lexeme_base = df_pt.pivot_table(
    index='user_id', 
    columns='lexeme_code', 
    values='user_ability_index',
    aggfunc='mean' # If there are multiple records for the same character, take the average
)

# Rename the columns to: lexeme_0, lexeme_1...
df_lexeme_base.columns = [f'lexeme_{int(col)}' for col in df_lexeme_base.columns]

# Handling missing values
df_lexeme_base = df_lexeme_base.fillna(0)
df_lexeme_base.head()

,lexeme_0,lexeme_1,lexeme_2,lexeme_3,lexeme_4,lexeme_5,lexeme_6,lexeme_7,lexeme_8,lexeme_9,...,lexeme_2805,lexeme_2806,lexeme_2807,lexeme_2808,lexeme_2809,lexeme_2810,lexeme_2811,lexeme_2812,lexeme_2813,lexeme_2814
user_id,,,,,,,,,,,,,,,,,,,,,
u:0X2,0.999994,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u:0b,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u:0xw,0.000000,0.0,0.0,0.0,0.000000,0.0,0.999937,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u:1EH,0.999992,0.0,0.0,0.0,0.999987,0.0,0.762756,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u:1gx,0.999994,0.0,0.0,0.0,0.000000,0.0,0.682692,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_user_fp = pd.merge(df_user_base, df_lexeme_base, on = 'user_id')
df_user_fp = pd.merge(df_user_fp, df_lexeme_seen, on = 'user_id')
df_user_fp

,user_id,max_history_seen,vocab_size,learning_speed,lexeme_0,lexeme_1,lexeme_2,lexeme_3,lexeme_4,lexeme_5,...,lexeme_2805_seen,lexeme_2806_seen,lexeme_2807_seen,lexeme_2808_seen,lexeme_2809_seen,lexeme_2810_seen,lexeme_2811_seen,lexeme_2812_seen,lexeme_2813_seen,lexeme_2814_seen
0,u:0X2,15,9,13.563969,0.999994,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,u:0b,12,12,3.629236,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,u:0xw,20,108,2.633296,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,u:1EH,14,52,2.580331,0.999992,0.0,0.0,0.0,0.999987,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,u:1gx,15,71,7.584663,0.999994,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,u:yT9,20,13,13.931693,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2705,u:yyO,21,14,2.044361,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2706,u:z4x,85,83,2.375734,0.958847,0.0,0.0,0.0,0.868058,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2707,u:zmi,12,42,8.380809,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_user_fp.to_csv('../data/users_fingerprint.csv', index=True)

## Data Normalization

In [12]:
from sklearn.preprocessing import StandardScaler

# Select the columns to be formalized. (排除 user_id)
cols_to_scale = ['max_history_seen', 'vocab_size', 'learning_speed']


scaler = StandardScaler()
df_scaled = df_user_fp.copy()
df_scaled[cols_to_scale] = scaler.fit_transform(df_user_fp[cols_to_scale])


print("The normalized characteristic distribution:")
print(df_scaled[cols_to_scale].describe().round(2))

The normalized characteristic distribution:
       max_history_seen  vocab_size  learning_speed
count           2709.00     2709.00         2709.00
mean               0.00       -0.00           -0.00
std                1.00        1.00            1.00
min               -0.67       -0.74           -1.43
25%               -0.48       -0.61           -0.87
50%               -0.31       -0.35           -0.39
75%                0.12        0.19            0.98
max               17.67       11.95            1.78


In [13]:
df_scaled.to_csv('../data/users_fingerprint_norm.csv', index=True)
df_scaled

,user_id,max_history_seen,vocab_size,learning_speed,lexeme_0,lexeme_1,lexeme_2,lexeme_3,lexeme_4,lexeme_5,...,lexeme_2805_seen,lexeme_2806_seen,lexeme_2807_seen,lexeme_2808_seen,lexeme_2809_seen,lexeme_2810_seen,lexeme_2811_seen,lexeme_2812_seen,lexeme_2813_seen,lexeme_2814_seen
0,u:0X2,-0.333881,-0.623280,1.458221,0.999994,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,u:0b,-0.405127,-0.580715,-0.690058,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,u:0xw,-0.215139,0.781374,-0.905419,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,u:1EH,-0.357630,-0.013178,-0.916872,0.999992,0.0,0.0,0.0,0.999987,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,u:1gx,-0.333881,0.256403,0.165261,0.999994,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,u:yT9,-0.215139,-0.566526,1.537738,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2705,u:yyO,-0.191391,-0.552338,-1.032770,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2706,u:z4x,1.328513,0.426664,-0.961114,0.958847,0.0,0.0,0.0,0.868058,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2707,u:zmi,-0.405127,-0.155062,0.337419,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
